In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.preprocessing import StandardScaler

## Importing the data

In [3]:
data = pd.read_csv('cal_housing_clean.csv')
data.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 7 columns):
housingMedianAge    20640 non-null float64
totalRooms          20640 non-null float64
totalBedrooms       20640 non-null float64
population          20640 non-null float64
households          20640 non-null float64
medianIncome        20640 non-null float64
medianHouseValue    20640 non-null float64
dtypes: float64(7)
memory usage: 1.1 MB


In [5]:
data.isnull().any()

housingMedianAge    False
totalRooms          False
totalBedrooms       False
population          False
households          False
medianIncome        False
medianHouseValue    False
dtype: bool

In [6]:
data.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


All columns are continuous.


## Exploratory Profiling

In [7]:
#!conda install -c conda-forge pandas_profiling
# Thank you for this awesome package
# https://github.com/pandas-profiling/pandas-profiling/graphs/contributors
from pandas_profiling import ProfileReport
# Note: problem with chesterish jupyter theme. reset to default. O0ps!

In [8]:
ProfileReport(data)

Number of variables,7
Number of observations,20640
Total Missing (%),0.0%
Total size in memory,1.1 MiB
Average record size in memory,56.0 B
Numeric,5
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,2


## Feature Selection

In [9]:
data = data.drop(['households', 'totalBedrooms'], axis=1) # removing features

In [10]:
data.head()

,housingMedianAge,totalRooms,population,medianIncome,medianHouseValue
0,41.0,880.0,322.0,8.3252,452600.0
1,21.0,7099.0,2401.0,8.3014,358500.0
2,52.0,1467.0,496.0,7.2574,352100.0
3,52.0,1274.0,558.0,5.6431,341300.0
4,52.0,1627.0,565.0,3.8462,342200.0


## Preprocessing

In [11]:
standardScaler = StandardScaler(with_mean=True, with_std=True)
standardScaler.fit(data)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [12]:
# Normal fit transform returns a numpy ndarray, so converting it to pd.df
columns = data.columns
indexes = data.index

In [13]:
scaledData = pd.DataFrame(standardScaler.transform(data), index=indexes, columns=columns)
scaledData.head()

,housingMedianAge,totalRooms,population,medianIncome,medianHouseValue
0,0.982143,-0.804819,-0.974429,2.344766,2.129631
1,-0.607019,2.045890,0.861439,2.332238,1.314156
2,1.856182,-0.535746,-0.820777,1.782699,1.258693
3,1.856182,-0.624215,-0.766028,0.932968,1.165100
4,1.856182,-0.462404,-0.759847,-0.012881,1.172900


## Train Test Split

In [14]:
x_data = scaledData.drop(['medianHouseValue'], axis=1)
x_data.head()
y_label = scaledData['medianHouseValue']
y_label.head()

0    2.129631
1    1.314156
2    1.258693
3    1.165100
4    1.172900
Name: medianHouseValue, dtype: float64

In [15]:
scaledData.head()

,housingMedianAge,totalRooms,population,medianIncome,medianHouseValue
0,0.982143,-0.804819,-0.974429,2.344766,2.129631
1,-0.607019,2.045890,0.861439,2.332238,1.314156
2,1.856182,-0.535746,-0.820777,1.782699,1.258693
3,1.856182,-0.624215,-0.766028,0.932968,1.165100
4,1.856182,-0.462404,-0.759847,-0.012881,1.172900


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_label, test_size=0.3, 
                                                    random_state = 0)

For DNNRegressor, we need

1. Feature column

2. Input Function

3. Model object

## Feature Columns

In [18]:
columns

Index(['housingMedianAge', 'totalRooms', 'population', 'medianIncome',
       'medianHouseValue'],
      dtype='object')

In [22]:
# I hope this can make feature_col vector wihtout manually typing all those names
feature_cols = []
for column in columns:
    if column!='medianHouseValue':
        feature_cols.append(tf.feature_column.numeric_column(column))
feature_cols

[NumericColumn(key='housingMedianAge', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='totalRooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='population', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='medianIncome', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Input Function

In [23]:
train_input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train,
                                                num_epochs=1000, shuffle=True)
test_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, y=y_test,
                                                     num_epochs=1, shuffle=False)

## Dense Neural Network Regressor Model

In [24]:
dnnRegressor = tf.estimator.DNNRegressor(hidden_units=[4, 5, 5], 
                                         feature_columns=feature_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpvfs0dinw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff08cbc62e8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training the DNN

In [25]:
dnnRegressor.train(input_fn=train_input_func, steps=1000)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpvfs0dinw/model.ckpt.
INFO:tensorflow:loss = 103.70192, step = 1
INFO:tensorflow:global_step/sec: 99.5329
INFO:tensorflow:loss = 55.164307, step = 101 (1.009 sec)
INFO:tensorflow:global_step/sec: 182.245
INFO:tensorflow:loss = 50.98037, step = 201 (0.546 sec)
INFO:tensorflow:global_step/sec: 153.755
INFO:tensorflow:loss = 58.243065, step =

## Evaluating the model

In [26]:
dnnRegressor.evaluate(input_fn=test_input_func, steps=None)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-18T10:38:34Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/tmpvfs0dinw/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-18-10:38:35
INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.44503897, global_step = 1000, label/mean = 0.0006307352, loss = 56.238396, prediction/mean = 0.00036508098
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmpvfs0dinw/model.ckpt-1000


{'average_loss': 0.44503897,
 'label/mean': 0.0006307352,
 'loss': 56.238396,
 'prediction/mean': 0.00036508098,
 'global_step': 1000}

## Making Predictions

In [30]:
predict_input_function = tf.estimator.inputs.pandas_input_fn(x=X_test, y=None,
                                                            num_epochs=1, shuffle=False)
predictions = dnnRegressor.predict(input_fn=predict_input_function)
predictionsList = list(predictions)
predictionsList

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvfs0dinw/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([-0.13668442], dtype=float32)},
 {'predictions': array([0.80127853], dtype=float32)},
 {'predictions': array([0.25624454], dtype=float32)},
 {'predictions': array([-0.7329405], dtype=float32)},
 {'predictions': array([0.49609095], dtype=float32)},
 {'predictions': array([-0.37940887], dtype=float32)},
 {'predictions': array([0.47986668], dtype=float32)},
 {'predictions': array([1.0987372], dtype=float32)},
 {'predictions': array([0.7192536], dtype=float32)},
 {'predictions': array([0.1441023], dtype=float32)},
 {'predictions': array([-0.08240128], dtype=float32)},
 {'predictions': array([-0.72281015], dtype=float32)},
 {'predictions': array([-0.6633817], dtype=float32)},
 {'predictions': array([-0.70895135], dtype=float32)},
 {'predictions': array([0.20220849], dtype=float32)},
 {'predictions': array([0.5281351], dtype=float32)},
 {'predictions': array([1.2723892], dtype=float32)},
 {'predictions': array([-0.5450579], dtype=float32)},
 {'predictions': array([-0.2

In [46]:
predictionsList[0]['predictions'][0]

IndexError: list index out of range

In [45]:
y_test[14740]

-0.606240636669762

In [38]:
assert(len(y_test) == len(predictionsList))
N = len(y_test)
sumSquaredError = 0
for i in range(N):
    sumSquaredError += 